In [1]:
import json
import numpy as np
import os
from PIL import Image
from pycocotools.coco import COCO
import shutil
import sys
import supervision as sv

In [2]:
sys.path.insert(1, '../..')

from data_processing.data_proc_lib.annotations import data_skeleton
from data_processing.data_proc_lib.transforms import do_paste
from data_processing.data_proc_lib.utilities import create_directories

This notebook will create annotations (masks and bounding boxes) for generated galaxies.

Set the following:

`THRESHOLD`: Pixel values below this value will be removed. This is to remove low levels of background noise that often appears surrounding images generated by GANs or VAEs. Recommended settings: 15 for VAE-generated galaxies, 25 for GAN-generated galaxies.

`category`: Which galaxy category we're annotating.

`OUTPUT_DIR`: Target output directory for annotated images.

`image_dir`: Input directory containing images (of a single category) to be annotated.

In [3]:
THRESHOLD = 15  # Use 25 for GAN, 15 for VAE
IMAGE_HEIGHT = 450
IMAGE_WIDTH = 450

In [ ]:
OUTPUT_DIR = "/mnt/data/rgn_ijcnn/augmented/vae_by_class"
create_directories(OUTPUT_DIR, split_by_class=True)
category = 3
image_dir = os.path.join(
    f"/home/matt/Documents/Project/repos/vae-torch-celeba/rand-cat-{category}-kld-0.05/generated/random", str(category),
    "train")
image_files = [f.name for f in os.scandir(image_dir) if not f.name.startswith('.')]

In [5]:
def create_anno_from_image(image_path, id):
    anno = {}
    segmentations = []

    image_array = np.array(Image.open(image_path))
    mask = np.logical_or(np.logical_or(image_array[:, :, 0] > THRESHOLD, image_array[:, :, 1] > THRESHOLD),
                         image_array[:, :, 2] > THRESHOLD)

    # Create polygons from transformed masks
    polygons = sv.mask_to_polygons(mask)
    for p in polygons:
        segmentations.append([float(s) for s in p.flatten()])

    anno["segmentation"] = segmentations

    tight_bbox_xyxy = sv.mask_to_xyxy(
        mask.reshape([1, IMAGE_HEIGHT, IMAGE_WIDTH]))
    width = int(tight_bbox_xyxy[0][2] - tight_bbox_xyxy[0][0])
    height = int(tight_bbox_xyxy[0][3] - tight_bbox_xyxy[0][1])

    tight_bbox_xywh = [
        int(tight_bbox_xyxy[0][0]),
        int(tight_bbox_xyxy[0][1]),
        width,
        height,
    ]

    transformed_bbox_area = tight_bbox_xywh[2] * tight_bbox_xywh[3]

    anno["num_keypoints"] = 0
    anno["area"] = transformed_bbox_area
    anno["iscrowd"] = 0
    anno["image_id"] = id + 1
    anno["bbox"] = tight_bbox_xywh
    #anno["keypoints"] = []
    anno["category_id"] = category + 1
    anno["id"] = id + 1

    return anno

In [6]:
def save_image(generated_image_path, output_dir):
    output_image_dir = os.path.join(output_dir, "train")
    output_image_file_path = os.path.join(output_image_dir, f"generated_{id + 1}.png")
    image = Image.open(generated_image_path)

    image = np.array(image).astype(np.uint8)

    offset = int((450 - 32) / 2)
    large_image_array = np.zeros((450, 450, 3), dtype=np.uint8)
    large_image_array[offset:(offset + 32), offset:(offset + 32), :] = image

    cleaned_image_array = do_paste(np.zeros((450, 450, 3), dtype=np.uint8), large_image_array, threshold=THRESHOLD)

    Image.fromarray(cleaned_image_array).save(output_image_file_path)

    return output_image_file_path

In [ ]:
original_annos = os.path.join("/mnt/data/rgn_ijcnn/cleaned", "annotations", "train.json")
augmented_data = data_skeleton(original_annos)
output_path = os.path.join(OUTPUT_DIR, str(category))
print(output_path)

for id, image in enumerate(image_files):
    generated_image_path = os.path.join(image_dir, image)
    output_image_path = save_image(generated_image_path, output_path)

    anno = create_anno_from_image(output_image_path, id)

    aug_image_data = {
        "file_name": f"generated_{id + 1}.png",
        "height": 450,
        "width": 450,
        "id": id + 1
    }

    # Append the image-related data to the annotation.
    augmented_data["images"].append(aug_image_data)

    augmented_data["annotations"].append(anno)

# Write the annotations file.
train_annos_filepath = os.path.join(output_path, "annotations", "train.json")

with open(train_annos_filepath, "w") as f:
    json.dump(augmented_data, f, ensure_ascii=False, indent=4)
